# problem formulation
* I have tabular dataset  called wish.com product and my goal to predict the product ratings given the other features known for a product on Wish.com but dataset is not clean and needs some of preprocessing  depending the model.
* we have 33 features  as input to dataset and output column called rating forin categories from 1 to 5.the data mining function that we need in this problems is classification and predictions.
* The challenges on business understanding,data selection,data preparation and
doing some preprocessing according to the model.

* The impact of the product on real life  help us to business understanding  , understand the customer base of the wish.com.  you can estimate how likely people will like your product. and the more the customers like the product. so we can get to **ideal solution** by knowing  the product that  is highly rated .so **technically**  : we apply life cycle of data mining to be able to predict the product that is highly rated.
 
* Data collection.
* Data selection.
* Data preparation.
* Modeling.
* Evaluation.
* Development.

 
 


# The experimental protocol that I applied it .
* it is hold out method that Indicates to spliting training data to train and test split  to be able to measure performance.
 Preprocessing steps that I need:
*   Inconsistent Data Entry.
*   Handling missing values.
*   Scaling and Normlization.
*   Inconsistent Data Entry.










# Plan for trial 1.
* The first step: I want to understand the data so I did Data visualization with excel and  I could determine the important features in data set(data extraction) .
* The second step:
I need to prepare data to do some pre processing because  the data is not clean.so I will drop useless columns, handle missing values and do normlization on data if the model need that.    
* the third step :  Build model
* the fourth step:   Evaluate model.
---
* In this try I worked for tunning hyperparamters especially class weights  to determine which features affected in the performance of the model, some preprocessing and made  feature slection manually.

**Observation** :
* SVM: I expectd  high performance and the model predict that .
* decision tree: I expectd high performance  and the result is increased as trees can choose equivalent splitting points without needing normlization or scaling.
* Naive bayes: I expectd low performance but f1_score increased. as it doesnot need pre processing






NOTES:I read  file .excel  and in the last line I converted it to csv. I did that because I want  try  the two ways of reading.

In [ ]:
! pip install fuzzywuzzy       # install fuzzy module.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Loading and Visualizing the Data
We first load the packages as usual.

In [ ]:

import pandas as pd
import sklearn.preprocessing as preprocessing
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import fuzzywuzzy
from fuzzywuzzy import process
import chardet


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Next, we load the data from  the excel file. It is important to remove the missing values.

In [ ]:
df = pd.read_excel ("/content/drive/MyDrive/train_new.xlsx",sheet_name='train_new',index_col="id")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 118 to 1223
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1094 non-null   float64
 1   retail_price                  1094 non-null   int64  
 2   units_sold                    1094 non-null   int64  
 3   uses_ad_boosts                1094 non-null   int64  
 4   rating_count                  1094 non-null   int64  
 5   badges_count                  1094 non-null   int64  
 6   badge_local_product           1094 non-null   int64  
 7   badge_product_quality         1094 non-null   int64  
 8   badge_fast_shipping           1094 non-null   int64  
 9   product_color                 1065 non-null   object 
 10  product_variation_size_id     1082 non-null   object 
 11  product_variation_inventory   1094 non-null   int64  
 12  shipping_option_name          1094 non-null   object 
 13  s

##Some of Data cleaning .
Handling missing value.

In [ ]:
df['rating'].value_counts()# check  the number of value 6 in column rating.

4    774
5    170
3    135
2     12
1      2
6      1
Name: rating, dtype: int64

In [ ]:
df['rating']=df['rating'].replace(6,5)#rating value from 1:5 so I repace the value 6 with value 5.

In [ ]:
df=df.drop(columns=['Positive_rate','origin_country', 'badge_local_product','badge_product_quality','badge_fast_shipping','product_variation_size_id','product_color','shipping_option_name'])

In [ ]:
df=df.fillna(0)

In [ ]:
df.isnull()#check nan values

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,product_variation_inventory,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,merchant_rating_count,merchant_rating,merchant_has_profile_picture,rating
id,,,,,,,,,,,,,,,,
118,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1317,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
562,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
240,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
999,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
809,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


##Manipulating the Data
We can get the shape of the data, which indicates the number of samples and number of attributes

In [ ]:
df.shape

(1094, 16)

In [ ]:
df.info()#print data frame information.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 118 to 1223
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1094 non-null   float64
 1   retail_price                  1094 non-null   int64  
 2   units_sold                    1094 non-null   int64  
 3   uses_ad_boosts                1094 non-null   int64  
 4   rating_count                  1094 non-null   int64  
 5   badges_count                  1094 non-null   int64  
 6   product_variation_inventory   1094 non-null   int64  
 7   shipping_option_price         1094 non-null   int64  
 8   shipping_is_express           1094 non-null   int64  
 9   countries_shipped_to          1094 non-null   int64  
 10  inventory_total               1094 non-null   int64  
 11  has_urgency_banner            1094 non-null   float64
 12  merchant_rating_count         1094 non-null   int64  
 13  m

In [ ]:
x=df.iloc[:,:-1]#features
y=df['rating']#labels

Next we split the thet raining data into training and test

In [ ]:
## Split into training and test

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=100) 


In [ ]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 820 entries, 1526 to 1067
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         820 non-null    float64
 1   retail_price                  820 non-null    int64  
 2   units_sold                    820 non-null    int64  
 3   uses_ad_boosts                820 non-null    int64  
 4   rating_count                  820 non-null    int64  
 5   badges_count                  820 non-null    int64  
 6   product_variation_inventory   820 non-null    int64  
 7   shipping_option_price         820 non-null    int64  
 8   shipping_is_express           820 non-null    int64  
 9   countries_shipped_to          820 non-null    int64  
 10  inventory_total               820 non-null    int64  
 11  has_urgency_banner            820 non-null    float64
 12  merchant_rating_count         820 non-null    int64  
 13  m

#Build  decision tree model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier( max_depth=3,max_features=5,min_samples_leaf=5,random_state=2,)#Tunning hyperpatamter manually.
clf = clf.fit(X_train,y_train)# # Fit a model 

In [ ]:
Y_pred=clf.predict(X_train)



# Measure other performance metrics

In [ ]:
print(f1_score(y_train, Y_pred, average = "weighted"))

0.6818568368382343


In [ ]:
#print("Accuracy:",metrics.accuracy_score(y_train, Y_pred))

In [ ]:
Y_test=clf.predict(X_test)


In [ ]:
print("f1_score:",metrics.f1_score(y_test, Y_test, average = "weighted"))

f1_score: 0.6608789921923139


#We can do the same operation with test data

In [ ]:
test_test=pd.read_excel("/content/drive/MyDrive/test_new 1 (1).xlsx",index_col="id")#read excel file.

In [ ]:
test_test= test_test.drop(columns=['badge_fast_shipping','tags','product_color','currency_buyer','badge_local_product','theme','crawl_month','origin_country','merchant_title','urgency_text','merchant_name','merchant_info_subtitle','product_variation_size_id','shipping_option_name','merchant_id','merchant_profile_picture','badge_product_quality'],axis=1)
# drop useless columns.

In [ ]:
test_test.isnull()#check nan value.

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,product_variation_inventory,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,merchant_rating_count,merchant_rating,merchant_has_profile_picture
id,,,,,,,,,,,,,,,
937,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
270,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
308,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
913,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
403,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1416,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
63,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
test_test = test_test.fillna(0)#fill nan value

In [ ]:
np.unique(test_test.isnull()) # check again

array([False])

#Manipulating the Data
We can get the shape of the data, which indicates the number of samples and number of attributes

In [ ]:
test_test.shape

(479, 15)

In [ ]:
test_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479 entries, 937 to 196
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         479 non-null    float64
 1   retail_price                  479 non-null    int64  
 2   units_sold                    479 non-null    int64  
 3   uses_ad_boosts                479 non-null    int64  
 4   rating_count                  479 non-null    int64  
 5   badges_count                  479 non-null    int64  
 6   product_variation_inventory   479 non-null    int64  
 7   shipping_option_price         479 non-null    int64  
 8   shipping_is_express           479 non-null    int64  
 9   countries_shipped_to          479 non-null    int64  
 10  inventory_total               479 non-null    int64  
 11  has_urgency_banner            479 non-null    float64
 12  merchant_rating_count         479 non-null    int64  
 13  mer

In [ ]:
test_pred=clf.predict(test_test)  # predict model
print(test_pred)
test_pred=test_pred.astype(float)#converting to float as i read from excel sheet  and by default data is integer
test_pred

[5 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 4 4 5 4 5 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4
 4 4 4 4 4 4 4 5 4 5 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 4 5 4 4 4
 4 4 4 4 4 4 5 4 4 4 4 4 5 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 5 4 5 4 4 4 4 4 4 4 4 5 4
 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 5 4 4 5 4 4 4 4
 4 4 4 4 4 4 4 4 5 4 4 4 5 4 4 4 5 4 4 4 4 4 4 4 5 4 4 4 5 4 4 4 4 4 5 5 4
 4 5 4 4 5 4 4 4 4 5 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4]


array([5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 5., 4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 5., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 5., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 5., 4., 4., 4.

#Saving prediction in file .csv

In [ ]:
_id = test_test.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred}))
dataframe.to_csv('dt_afaf.csv', index=False)

#Build Support vector machine model.
 With Support vector machine, we need apply normlization

In [ ]:

#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(C=.1,kernel="rbf",gamma="scale") # Linear Kernel,C- It is a hypermeter in SVM to control error,#  gamma as grid search


In [ ]:


#scaling
scaler = preprocessing.StandardScaler().fit(X_train
    )

# mean values for each column
print(scaler.mean_)

[8.32429268e+00 2.39463415e+01 4.56656341e+03 4.36585366e-01
 9.19668293e+02 1.14634146e-01 3.28134146e+01 2.35731707e+00
 3.65853659e-03 4.02085366e+01 4.98756098e+01 3.17073171e-01
 2.53568366e+04 4.03577481e+00 1.56097561e-01]


In [ ]:
X_train_preprocessed = scaler.transform(
    X_train)

# zero mean (or very close to zero) for each column
print('scaled data should have zero mean', X_train_preprocessed.mean(axis=0))

scaled data should have zero mean [-6.17392316e-17  3.46606213e-17 -3.03280436e-17 -4.33257766e-18
  1.29977330e-17 -8.66515531e-17  5.41572207e-17 -8.99009864e-17
  0.00000000e+00 -1.16979597e-16  2.39374916e-16 -2.16628883e-18
 -8.66515531e-18 -9.25546902e-16 -5.63235095e-17]


In [ ]:
# zero std (or very close to 1) for each column
print('scaled data should have 1 std',X_train_preprocessed.std(axis=0))

scaled data should have 1 std [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:


#scaling
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(X_train
    )
# here we specified the range as 0 to 1

In [ ]:
X_train_preprocessed = scaler.transform(
    X_train)#X_train_preprocessed after scaling. on X_train.

# zero min for each column
print(X_train_preprocessed.min(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# max one for each column
print(X_train_preprocessed.max(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
#Apply normlizaion.


scaler = preprocessing.Normalizer(norm='l2').fit(
    X_train)
X_train_preprocessed= scaler.transform(
    X_train )


In [ ]:
# unit l2 norm
from numpy import linalg as LA
LA.norm(X_train_preprocessed, ord=2, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
clf.fit(X_train_preprocessed,y_train)# fit model

SVC(C=0.1)

In [ ]:
Y_pred=clf.predict(X_train_preprocessed)# prediction of model.


#Measure other performance metrics

In [ ]:
print(f1_score(y_train, Y_pred, average="weighted"))

0.5876656047074389


In [ ]:
print("f1_score:",metrics.f1_score(y_train, Y_pred, average="weighted"))

f1_score: 0.5876656047074389


In [ ]:
Y_test=clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
print("f1_score:",metrics.f1_score(y_test,Y_test , average="weighted"))

f1_score: 0.5822066615608247


In [ ]:
test_pred=clf.predict(test_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
test_pred=test_pred.astype(float)#converting to float as i read from excel sheet  and by default data is integer
test_pred

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

#saving result of prediction in file .csv

In [ ]:
_id = test_test.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred}))
dataframe.to_csv('svm_afaf.csv', index=False)

#Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB# import GaussianNB 
#Create a Gaussian Classifier
clf= GaussianNB()
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}#Tunning hyperparamters.

In [ ]:
clf = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
clf.fit(X_train,y_train)#fit model

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


GridSearchCV(cv=10, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.848035...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [ ]:
Y_pred=clf.predict(X_train)#prediction of model

#Measure other performance metrics

In [ ]:
print(f1_score(y_train, Y_pred, average="weighted"))

print("f1_score:",metrics.f1_score(y_test,Y_test, average="weighted" ))

0.5876656047074389
f1_score: 0.5822066615608247


In [ ]:
test_pred=test_pred.astype(float)#converting to float as i read from excel sheet  and by default data is integer
test_pred

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

#Saving result of prediction in file.csv

In [ ]:
_id = test_test.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred}))
dataframe.to_csv('naive_afaf.csv', index=False)

# **Trail_2** 
 In this try I used different features, all types of data cleaning and also different hyper paramters.
1. I inserted inconsistent Data Entry in python like  columns[tags,'product_color,product_variation_size_id,shipping_name}   
2.  Idid  data cleaning with applying  the fuzzywuzzy package to help identify which strings are closest to each other.
3. Handling missing values. 
4.  Scaling &Normlization.
---
* Using label encoder to convert categorical value to numerical .
 
Observation:
* . SVM:   I expect high performance because  it needs to normlization and scaling.
* . decision tree:  I expectd low performance but  the result is increased as trees can choose equivalent splitting points without needing normlization or scaling.
* .  Naive bayes: I expectd low performance  and exactly  f1_score decreased. as it doesnot need pre processing.












#Loading and Visualizing the Data
We first load the packages as usual.

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/train_new.csv",index_col="id")#read data from train_new.csv.

In [ ]:
print(df1["product_variation_size_id"])#inconsistent column and its type categorical.

id
272       M
1515      L
940       L
309      XS
226       S
       ... 
1381      S
212     H01
163       M
674       M
1536     XS
Name: product_variation_size_id, Length: 1094, dtype: object


In [ ]:
df1["product_variation_size_id"].drop_duplicates(inplace=True)# drop dublicate values.


In [ ]:
df1["product_variation_size_id"].dtypes# determine type of  column.

dtype('O')

In [ ]:
words = df1["product_variation_size_id"].str.split()
words
word_counts = pd.value_counts(words.apply(pd.Series).stack())#???determine numberof values of each word in product_variation_size_id.
word_counts

S           453
XS          250
M           137
XXS          78
L            30
           ... 
US-S          1
pants-S       1
XXXXL         1
SIZE-XXS      1
H01           1
Length: 100, dtype: int64

#Drop values  that repeat only one.

In [ ]:
df1.drop(df1.index[df1["product_variation_size_id"] == 'US-S'], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'pants-S '], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'XXXXL '], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'SIZE-XXS'], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'size'], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'of'], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'Round'], inplace=True)
df1.drop(df1.index[df1["product_variation_size_id"] == 'H01'], inplace=True)



In [ ]:
product_variation_size_id=df1["product_variation_size_id"].unique()# print unique values
product_variation_size_id

array(['M', 'L', 'XS', 'S', 'XL', '26(Waist 72cm 28inch)', 'S.',
       'S(bust 88cm)', 'XXS', 's', '29', nan, 'choose a size', 'XXXS',
       'Base Coat', 'Size M', 'XXL', 'M.', 'XS.',
       '100 x 100cm(39.3 x 39.3inch)', '2pcs', '4XL', '1', '25-S',
       'Size-XXS', '5PAIRS', '35', 'Pack of 1', 'Size S', 'Size-S', '6XL',
       '25', 'S/M(child)', '60', 'Size-XS', 'S (waist58-62cm)',
       'SIZE XXS', '10 ml', 'X   L', 'Women Size 36', '04-3XL',
       'Size -XXS', '1 pc.', 'Floating Chair for Kid', 'S Pink', '34',
       'Size XXS', 'pants-S', 'XXXXL', 'SIZE XS', '1pc', 'Size S.',
       '100 cm', 'S..', '4-5 Years', '5', '33', '30 cm', '2', 'XXXXXL',
       '20PCS-10PAIRS', '2XL', 'Size-5XL', 'Size4XL', 'One Size',
       'size S', 'Size/S', 'B', 'SizeL', '20pcs', '1 PC - XL', 'Suit-S',
       'Base & Top & Matte Top Coat', 'Baby Float Boat', '1m by 3m',
       'SIZE S', 'White', '40 cm', '5XL', '10pcs'], dtype=object)

In [ ]:
product_variation_size_id=df1['product_variation_size_id'] = df1['product_variation_size_id'].fillna(0)#to fill nan value in the column.


In [ ]:

df1['product_variation_size_id'] = df1['product_variation_size_id'].astype(str).str.replace('.', '')

df1['product_variation_size_id']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


id
272      M
1515     L
940      L
309     XS
226      S
        ..
23       S
1381     S
163      M
674      M
1536    XS
Name: product_variation_size_id, Length: 1090, dtype: object

In [ ]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):#fuzzy function to clean inconsistent columns
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
     # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_variation_size_id', string_to_match="s")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_variation_size_id', string_to_match="M")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_variation_size_id', string_to_match="XS")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_variation_size_id', string_to_match="L")


All done!


In [ ]:
df1["product_variation_size_id"]

id
272      M
1515     L
940      L
309     XS
226     XS
        ..
23      XS
1381    XS
163      M
674      M
1536    XS
Name: product_variation_size_id, Length: 1090, dtype: object

In [ ]:
# get all the unique values in the 'Country' column
product_variation_size_idd = df1['product_variation_size_id'].unique()

# sort them alphabetically and then take a closer look
product_variation_size_idd.sort()
product_variation_size_idd

array(['0', '04-3XL', '1', '1 PC - XL', '1 pc', '10 ml', '100 cm',
       '100 x 100cm(393 x 393inch)', '10pcs', '1m by 3m', '1pc', '2',
       '20PCS-10PAIRS', '20pcs', '25', '25-S', '26(Waist 72cm 28inch)',
       '29', '2pcs', '30 cm', '33', '34', '35', '4-5 Years', '40 cm', '5',
       '5PAIRS', '60', 'B', 'Baby Float Boat',
       'Base & Top & Matte Top Coat', 'Base Coat',
       'Floating Chair for Kid', 'L', 'M', 'One Size', 'Pack of 1',
       'S (waist58-62cm)', 'S Pink', 'S(bust 88cm)', 'S/M(child)',
       'SIZE S', 'SIZE XS', 'SIZE XXS', 'Size -XXS', 'Size M', 'Size S',
       'Size XXS', 'Size-5XL', 'Size-S', 'Size-XS', 'Size-XXS', 'Size/S',
       'Size4XL', 'SizeL', 'Suit-S', 'White', 'Women Size 36', 'XS',
       'XXXXL', 'XXXXXL', 'choose a size', 'pants-S', 'size S'],
      dtype=object)

In [ ]:
df1["product_variation_size_id"].shape

(1090,)

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting categorical to numerical value.
df1['product_variation_size_id'] = le.fit_transform(df1.product_variation_size_id.values)
df1['product_variation_size_id'].dtypes

dtype('int64')

In [ ]:
print(df1["shipping_option_name"].shape)

(1090,)


In [ ]:
c=df1["shipping_option_name"].drop_duplicates(inplace=True)
print(c)

None


In [ ]:
words = df1["shipping_option_name"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

Livraison                  1046
standard                   1045
Standard                     20
Shipping                     16
Envio                         6
Padrão                        6
الشحن                         4
القياسي                       4
Expediere                     4
Envío                         3
normal                        3
Standardversand               3
Gönderi                       2
Стандартная                   2
Express                       2
wysyłka                       2
Standart                      2
Standardowa                   2
доставка                      2
Spedizione                    1
ការដឹកជញ្ជូនតាមស្តង់ដារ       1
dtype: int64

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='shipping_option_name', string_to_match="Livraison ")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='shipping_option_name', string_to_match=" Standard")


All done!


In [ ]:
# get all the unique values in the 'Country' column
shipping_option_namee= df1['shipping_option_name'].unique()

# sort them alphabetically and then take a closer look
shipping_option_namee.sort()
shipping_option_namee

array([' Standard', 'Envio Padrão', 'Envío normal', 'Livraison ',
       'Стандартная доставка', 'الشحن القياسي', 'ការដឹកជញ្ជូនតាមស្តង់ដារ'],
      dtype=object)

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting categorical value to numerical value.
df1['shipping_option_name'] = le.fit_transform(df1.product_color.values)
df1['shipping_option_name'].dtypes

dtype('int64')

In [ ]:
df1['product_color'] = df1['product_color'].astype(str).str.replace('product_color', '')

product_variation_size_id=df1["product_color"].unique()
product_variation_size_id

array(['yellow', 'black', 'white', 'lakeblue', 'apricot', 'brown',
       'winered', 'blue', 'red', 'navyblue', 'green', 'khaki', 'White',
       'white & green', 'multicolor', 'lightpink', 'pink', 'RED',
       'armygreen', 'lightblue', 'nan', 'coffee', 'grey', 'skyblue',
       'watermelonred', 'pink & black', 'whitefloral', 'purple', 'navy',
       'pink & white', 'rosered', 'orange', 'Black', 'mintgreen',
       'leopardprint', 'gray', 'navy blue', 'star', 'rose', 'lightyellow',
       'camouflage', 'black & yellow', 'whitestripe', 'navyblue & white',
       'black & blue', 'lightred', 'violet', 'gold', 'black & green',
       'white & black', 'burgundy', 'black & white', 'lightgrey',
       'coolblack', 'lightgreen', 'beige', 'darkblue', 'darkgreen',
       'silver', 'wine red', 'Army green', 'rainbow', 'claret', 'floral',
       'brown & yellow', 'light green', 'Pink', 'blue & pink',
       'dustypink', 'camel', 'orange-red', 'rosegold', 'ivory',
       'fluorescentgreen', 'winer

In [ ]:
print(df1["product_color"].shape)

(1090,)


In [ ]:
df1["product_color"].drop_duplicates(inplace=True) #drop duplicates.


In [ ]:
words = df1["product_color"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())     #determine number of words to each value.
word_counts

black          233
white          189
blue            82
green           75
pink            74
              ... 
darkgreen        1
star             1
whitestripe      1
lightgrey        1
lightyellow      1
Length: 73, dtype: int64

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="black")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="red")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="lakeblue")


All done!


In [ ]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=df1, column='product_color', string_to_match="white")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="apricot")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="purple")


All done!


In [ ]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=df1, column='product_color', string_to_match="blue")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="green")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='product_color', string_to_match="pink")


All done!


In [ ]:
# get all the unique values in the 'Country' column
product_colors= df1['product_color'].unique()

# sort them alphabetically and then take a closer look
product_colors.sort()
product_colors

array(['apricot', 'army', 'beige', 'black', 'blue', 'brown',
       'brown & yellow', 'burgundy', 'camel', 'camouflage', 'claret',
       'coffee', 'floral', 'gold', 'gray', 'green', 'ivory', 'khaki',
       'leopard', 'lightgray', 'lightgrey', 'lightyellow', 'multicolor',
       'nan', 'navy', 'orange', 'orange-red', 'pink', 'purple', 'rainbow',
       'silver', 'star', 'violet', 'watermelonred', 'white',
       'winered & yellow', 'yellow'], dtype=object)

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()# converting categorical value to numerical valu.
df1['product_color'] = le.fit_transform(df1.product_color.values)
df1['product_color'].dtypes

dtype('int64')

In [ ]:
print(df1["tags"])

id
272     Summer,soildcolor,Plus Size,Tank,camisole,Tops...
1515    bathing suit,Plus Size,bikini set,sexy swimsui...
940     Summer,Vest,momshirt,Get,summer t-shirts,funny...
309     Summer,Shorts,pants,Beach,Plus Size,beachpant,...
226     Summer,Floral print,women dresses,fashion dres...
                              ...                        
23      Summer,Vest,Plus Size,summer t-shirts,tank top...
1381    corset top,Fashion,crop top,camisole,knittop,T...
163     Summer,Vest,Fashion,Tank,camisole,Halter,Beaut...
674     Summer,Shorts,Cotton,Cotton T Shirt,letter pri...
1536    sleeve v-neck,Summer,Plus Size,Sleeve,Dresses,...
Name: tags, Length: 1090, dtype: object


In [ ]:
l=df1["tags"].drop_duplicates(inplace=True)     # drop duplicated values.
print(l)

None


In [ ]:
words = df1["tags"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

Summer,Plus                                        136
&                                                  104
dress,Women's                                       88
dress                                               88
sleeve                                              55
                                                  ... 
top,Lace,Hollow-out,Summer,lace                      1
top,Tops,slashneck,flaresleeve,clubwear,Women's      1
Fashion,Fashion,Embroidery,summer                    1
clothes                                              1
color,knee,Fashion,Casual                            1
Length: 4327, dtype: int64

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='tags', string_to_match="Summer,Plus ")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='tags', string_to_match="&")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='tags', string_to_match="dress,Women's ")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=df1, column='tags', string_to_match="sleeve  ")

All done!


In [ ]:
# get all the unique values in the 'Country' column
tagss= df1['tags'].unique()

# sort them alphabetically and then take a closer look
tagss.sort()
tagss

array(["2 Piece Swimsuit,Plus Size,Swimming,plus size bikinis,sexy swimsuit,Bikini swimwear,women swimsuit,Beach,sexy bikini,Swimwear,Halter,Suits,Bikini,Fashion,Summer,swimsuits for girls,bandage bikini,Halter Bikini,Swimsuit,Women's Fashion,Women",
       '2pieceset,Shorts,crop top,Sleeve,Summer,tiktok,Spring,Tops,Spaghetti Strap,short sleeves,Fashion',
       "2pieceset,short dress,Skirts,Tops,slim,short sleeves,V-neck,bodycon,clubwear,Women's Fashion,party,croptopandskirt,sexy,Mini dress,Lace,Dress,Women",
       'Beds,Outdoor,inflatablefloatchair,Sports & Outdoors,Home & Living,Inflatable,cushionbed,swimmingpoolseat,Seats,water,hammock,pool,swimmingchair,Swimming,toolsgadget,mattress,Chair,Furniture,Watersports',
       "Bikinis Set,Womens Swimsuit,Halter,Bikini swimwear,women swimsuit,padded bikini,bandage bikini,Swimwear,swimsuits for women,Bikini,Suits,Women's Fashion,Fashion,Womens swimwear,high waist bikini,Halter Bikini,Swimsuit,Beach,Swimming,Women",
       "Bikinis Set,wom

##Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting from categorical to numeric value.
df1['tags'] = le.fit_transform(df1.tags.values)
df1['tags'].dtypes

dtype('int64')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 272 to 1536
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1090 non-null   float64
 1   retail_price                  1090 non-null   int64  
 2   currency_buyer                1090 non-null   object 
 3   units_sold                    1090 non-null   int64  
 4   uses_ad_boosts                1090 non-null   int64  
 5   rating                        1090 non-null   float64
 6   rating_count                  1090 non-null   int64  
 7   badges_count                  1090 non-null   int64  
 8   badge_local_product           1090 non-null   int64  
 9   badge_product_quality         1090 non-null   int64  
 10  badge_fast_shipping           1090 non-null   int64  
 11  tags                          1090 non-null   int64  
 12  product_color                 1090 non-null   int64  
 13  p

In [ ]:
df1.isnull()

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,theme,crawl_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
272,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
1515,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False
940,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False
309,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False
226,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1381,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False
163,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False


In [ ]:
df1['rating']=df1['rating'].replace(6,5)

In [ ]:
df1['rating']

id
272     4.0
1515    4.0
940     4.0
309     3.0
226     4.0
       ... 
23      4.0
1381    4.0
163     4.0
674     4.0
1536    3.0
Name: rating, Length: 1090, dtype: float64

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 272 to 1536
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1090 non-null   float64
 1   retail_price                  1090 non-null   int64  
 2   currency_buyer                1090 non-null   object 
 3   units_sold                    1090 non-null   int64  
 4   uses_ad_boosts                1090 non-null   int64  
 5   rating                        1090 non-null   float64
 6   rating_count                  1090 non-null   int64  
 7   badges_count                  1090 non-null   int64  
 8   badge_local_product           1090 non-null   int64  
 9   badge_product_quality         1090 non-null   int64  
 10  badge_fast_shipping           1090 non-null   int64  
 11  tags                          1090 non-null   int64  
 12  product_color                 1090 non-null   int64  
 13  p

In [ ]:
df1=df1.drop(columns=['currency_buyer','urgency_text','origin_country','merchant_title','merchant_name','merchant_info_subtitle','merchant_has_profile_picture','merchant_profile_picture','merchant_id','theme','crawl_month'],axis=1)
#drop usless column

In [ ]:
df1.shape

(1090, 22)

In [ ]:
df1=df1.fillna(0)#fill nan value in the column.
df1.isnull()
print(df1["product_variation_size_id"])

id
272     34
1515    33
940     33
309     58
226     58
        ..
23      58
1381    58
163     34
674     34
1536    58
Name: product_variation_size_id, Length: 1090, dtype: int64


In [ ]:
x=df1.iloc[:,:-1]
y=df1['rating']
y

id
272     4.0
1515    4.0
940     4.0
309     3.0
226     4.0
       ... 
23      4.0
1381    4.0
163     4.0
674     4.0
1536    3.0
Name: rating, Length: 1090, dtype: float64

In [ ]:
df1.shape

(1090, 22)

In [ ]:
x.shape

(1090, 21)

#Spliting training data to train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=100) 


In [ ]:
from sklearn.tree import DecisionTreeClassifier#import model.

clf1 = DecisionTreeClassifier(criterion='gini',splitter='best', max_depth=5,max_features=7)# Tunning hyper paramters.
clf1 = clf1.fit(X_train,y_train)

In [ ]:
Y_pred=clf1.predict(X_train)

In [ ]:
#print(f1_score(y_train, Y_pred, average = "weighted"))

In [ ]:
print("f1_score:",metrics.f1_score(y_train, Y_pred, average = "weighted"))

f1_score: 0.7905427944234746


In [ ]:
Y_test=clf1.predict(X_test)

In [ ]:
print("f1_score:",metrics.f1_score(y_test,Y_test, average = "weighted" ))

f1_score: 0.7407276526003576


In [ ]:
test_test1=pd.read_csv("/content/test_new.csv",index_col="id")

In [ ]:
test_test1.head()

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,theme,crawl_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
937,5.75,5,EUR,10,1,0,0,0,0,0,"Funny,letter print,Fashion,Shirt,Sleeve,Summer...",white,XS,5,Livraison standard,2,0,139,50,NaN,NaN,CN,liyang163,liyang163,33 % avis positifs (3 notes),3,2.333333,5e63469b2fdc774466e15dd5,0,NaN,summer,2020-08
270,7.00,7,EUR,5000,0,579,0,0,0,0,"Women Vest,Vest,Fashion,Women Blouse,long dres...",black,XS,50,Livraison standard,2,0,31,50,NaN,NaN,CN,singing the song,singingthesong,"86 % avis positifs (4,213 notes)",4213,4.058391,583547f417dc6224cb464117,0,NaN,summer,2020-08
308,5.00,16,EUR,1000,1,62,0,0,0,0,"hollowouttanktop,Tanktops for women,Plus Size,...",blue,XS,20,Livraison standard,2,0,41,50,NaN,NaN,CN,Surper boutique,surperboutique,"85 % avis positifs (10,501 notes)",10501,4.029235,57086fd7fa6bee434d0e5852,0,NaN,summer,2020-08
913,18.00,27,EUR,5000,1,1903,0,0,0,0,"men jeans,Shorts,Summer,Vintage,Denim,Men,casu...",blue,M,50,Livraison standard,5,0,42,50,NaN,NaN,CN,postonny,postonny,"86 % avis positifs (15,855 notes)",15855,4.052917,5a17b55d1f4d8c4f0b44b0ce,0,NaN,summer,2020-08
403,12.00,81,EUR,20000,0,1799,0,0,0,0,"Shorts,Lace,pants,Dress,Women Leggings,Hot pan...",white,M,50,Livraison standard,3,0,38,50,NaN,NaN,CN,fashionforgirls,guangzhouchanny,"88 % avis positifs (151,914 notes)",151914,4.127921,53aa664438d3046ee44a5024,0,NaN,summer,2020-08


In [ ]:
r=test_test1["product_variation_size_id"].drop_duplicates(inplace=True)
print(r)

None


In [ ]:
words =test_test1 ["product_variation_size_id"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

S             200
XS            110
M              65
XXS            24
L              20
             ... 
CM              1
100pcs          1
first           1
generation      1
One             1
Length: 70, dtype: int64

In [ ]:

test_test1['product_variation_size_id'] = test_test1['product_variation_size_id'].astype(str).str.replace('.', '')

product_variation_size_id=test_test1["product_variation_size_id"].unique()
product_variation_size_id

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


array(['XS', 'M', 'XXS', 'S', 'L', 'S(Pink & Black)', 'S(bust 88cm)',
       'SizeL', 'SIZE-XXS', 'XL', '1', '32/L', 'daughter 24M', 'XXXL',
       '5XL', '4', '1 pc', '4XL', 'Size S', '2XL', 's', '3XL',
       '80 X 200 CM', 'EU 35', '34', 'XXXXL', 'XXL', '100pcs', 'SIZE XS',
       'first  generation', '2pcs', 'Size--S', 'SIZE-4XL', 'Suit-S',
       'Women Size 37', 'S Diameter 30cm', 'XXXS', 'Size-L',
       'AU plug Low quality', '3 layered anklet', '17', 'size S',
       'US 65 (EU 37)', 'US55-EU35', 'EU39(US8)', 'nan', '36', '33',
       'One Size'], dtype=object)

In [ ]:
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == 'CM '], inplace=True)
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == '100pcs'], inplace=True)
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == 'XXXXL '], inplace=True)
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == 'first'], inplace=True)
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == 'generation'], inplace=True)
product_variation_size_id=test_test1.drop(df1.index[df1["product_variation_size_id"] == 'On'], inplace=True)
#product_variation_size_id=df1.drop(df1.index[df1["product_variation_size_id"] == 'Round'], inplace=True)
#product_variation_size_id=df1.drop(df1.index[df1["product_variation_size_id"] == 'H01'], inplace=True)
df1["product_variation_size_id"].unique()


array([34, 33, 58, 16, 39, 17,  0, 61, 31, 45,  7, 18,  2, 15, 51, 26, 22,
       36, 46, 49, 14, 40, 27, 50, 37, 43,  5, 57,  1, 44,  4, 32, 38, 21,
       47, 62, 59, 42, 10,  6, 23, 25, 20, 19, 11, 60, 12, 48, 53, 35, 63,
       52, 28, 54, 13,  3, 55, 30, 29,  9, 41, 56, 24,  8])

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_variation_size_id', string_to_match="s")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_variation_size_id', string_to_match="M")



All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_variation_size_id', string_to_match="XS")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_variation_size_id', string_to_match="L")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_variation_size_id', string_to_match="XXS ")


All done!


In [ ]:
# get all the unique values in the 'Country' column
product_variation_size_idd = test_test1['product_variation_size_id'].unique()

# sort them alphabetically and then take a closer look
product_variation_size_idd.sort()
product_variation_size_idd

array(['1', '1 pc', '100pcs', '17', '2pcs', '3 layered anklet', '32/L',
       '33', '34', '36', '4', '80 X 200 CM', 'AU plug Low quality',
       'EU 35', 'EU39(US8)', 'L', 'M', 'One Size', 'S Diameter 30cm',
       'S(Pink & Black)', 'S(bust 88cm)', 'SIZE XS', 'SIZE-4XL', 'Size S',
       'Size--S', 'Size-L', 'SizeL', 'Suit-S', 'US 65 (EU 37)',
       'US55-EU35', 'Women Size 37', 'XXS ', 'daughter 24M',
       'first  generation', 'nan', 'size S'], dtype=object)

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting categorical to numircal.
test_test1['product_variation_size_id'] = le.fit_transform(test_test1.product_variation_size_id.values)
test_test1['product_variation_size_id'].dtypes

dtype('int64')

In [ ]:
test_test1["shipping_option_name"].drop_duplicates(inplace=True)
print(test_test1["shipping_option_name"].shape)

(479,)


In [ ]:
words = test_test1["shipping_option_name"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

standard               462
Livraison              461
Standard                 7
Shipping                 5
Envio                    3
Padrão                   3
wysyłka                  2
Envío                    2
normal                   2
การส่งสินค้ามาตรฐาน      2
Expediere                2
Standardowa              1
Spedizione               1
Ekspresowa               1
Стандартная              1
доставка                 1
dtype: int64

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='shipping_option_name', string_to_match="Livraison ")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='shipping_option_name', string_to_match="Standard ")


All done!


In [ ]:
# get all the unique values in the 'Country' column
shipping_option_namee= test_test1['shipping_option_name'].unique()

# sort them alphabetically and then take a closer look
shipping_option_namee.sort()
shipping_option_namee

array(['Ekspresowa wysyłka', 'Envio Padrão', 'Envío normal', 'Livraison ',
       'Standard ', 'Стандартная доставка', 'การส่งสินค้ามาตรฐาน'],
      dtype=object)

In [ ]:
le = preprocessing.LabelEncoder()
test_test1['shipping_option_name'] = le.fit_transform(test_test1.product_variation_size_id.values)
test_test1['shipping_option_name'].dtypes

dtype('int64')

In [ ]:
test_test1['product_color'] = test_test1['product_color'].astype(str).str.replace('product_color', '')

product_variation_size_id=test_test1["product_color"].unique()
product_variation_size_id

array(['white', 'black', 'blue', 'yellow', 'winered', 'navyblue', 'gray',
       'grey', 'nan', 'green', 'pink', 'armygreen', 'red',
       'black & green', 'lightblue', 'pink & blue', 'orange', 'rosered',
       'purple', 'watermelonred', 'coralred', 'apricot', 'multicolor',
       'tan', 'orange & camouflage', 'white & green', 'army green',
       'brown', 'silver', 'offblack', 'rose', 'jasper', 'beige', 'Pink',
       'floral', 'lightyellow', 'white & red', 'red & blue', 'khaki',
       'camel', 'leopard', 'camouflage', 'greysnakeskinprint',
       'fluorescentgreen', 'coffee', 'lightpurple', 'black & stripe',
       'lightgreen', 'darkblue', 'navy', 'wine', 'lightkhaki',
       'pink & grey', 'prussianblue', 'skyblue', 'gray & white',
       'orange-red'], dtype=object)

In [ ]:
test_test1["product_color"].drop_duplicates(inplace=True)
print(df1["product_color"])

id
272     36
1515     3
940     34
309      4
226      0
        ..
23      15
1381    15
163     28
674      3
1536    36
Name: product_color, Length: 1090, dtype: int64


In [ ]:
words = test_test1["product_color"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

white                 85
black                 85
yellow                36
red                   34
green                 33
pink                  33
blue                  24
grey                  24
purple                16
orange                12
nan                   12
&                     10
navyblue              10
armygreen              9
beige                  9
winered                9
gray                   6
multicolor             6
lightblue              4
rosered                3
floral                 3
skyblue                2
darkblue               2
fluorescentgreen       2
leopard                2
khaki                  2
brown                  2
camouflage             2
coralred               2
offblack               1
coffee                 1
watermelonred          1
prussianblue           1
lightkhaki             1
wine                   1
navy                   1
lightgreen             1
stripe                 1
lightpurple            1
apricot                1


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="black")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="red")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="lakeblue")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="white")


All done!


In [ ]:
# use the function we just wrote to replace 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="apricot")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="purple")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="grey")


All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='product_color', string_to_match="orange")


All done!


In [ ]:
# get all the unique values in the 'Country' column
product_colors= test_test1['product_color'].unique()

# sort them alphabetically and then take a closer look
product_colors.sort()
product_colors

array(['Pink', 'apricot', 'army green', 'armygreen', 'beige', 'black',
       'brown', 'camel', 'coffee', 'fluorescentgreen', 'grey',
       'greysnakeskinprint', 'jasper', 'khaki', 'lakeblue', 'leopard',
       'lightgreen', 'lightkhaki', 'lightyellow', 'multicolor', 'nan',
       'navy', 'orange', 'pink', 'purple', 'silver', 'tan',
       'watermelonred', 'white', 'yellow'], dtype=object)

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting  categorical to numeric values.
test_test1['product_color'] = le.fit_transform(test_test1.product_color.values)
test_test1['product_color'].dtypes

dtype('int64')

In [ ]:
test_test1["tags"].drop_duplicates(inplace=True)
print(test_test1["tags"])

id
937     Funny,letter print,Fashion,Shirt,Sleeve,Summer...
270     Women Vest,Vest,Fashion,Women Blouse,long dres...
308     hollowouttanktop,Tanktops for women,Plus Size,...
913     men jeans,Shorts,Summer,Vintage,Denim,Men,casu...
403     Shorts,Lace,pants,Dress,Women Leggings,Hot pan...
                              ...                        
1166    mensummertshirt,Summer,Fashion,Men's Fashion,S...
1416    blouse,Summer,Vest,Plus Size,letter print,cute...
63      blouse,Summer,Plus Size,Tops & Blouses,Shirt,W...
626     Summer,slideslipper,Sandals,fur,Indoor,Fox,Ind...
196     Summer,Plus Size,2pieceset,Women,slim,T Shirts...
Name: tags, Length: 479, dtype: object


In [ ]:
words = test_test1["tags"].str.split()
word_counts = pd.value_counts(words.apply(pd.Series).stack())
word_counts

&                                                                 63
Summer,Plus                                                       61
dress                                                             46
dress,Women's                                                     36
for                                                               34
                                                                  ..
shoes,sandalendamen,Sandals,Fashion,Womens                         1
Fashion,Fashion,Casual,summerjumpsuit,sleeveless,Rompers,women     1
Suits,shortjumpsuit,Women's                                        1
Strap,One-Piece                                                    1
Dye,Dyes,sexy,shortsforwomen,Outfits,Fashion,Women                 1
Length: 2498, dtype: int64

In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='tags', string_to_match="Summer,Plus ")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='tags', string_to_match="& ")

All done!


In [ ]:
# use the function we just wrote to replace close matches  
replace_matches_in_column(df=test_test1, column='tags', string_to_match="dress,Women's  ")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='tags', string_to_match="dress")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='tags', string_to_match="sleeve ")

All done!


In [ ]:
# use the function we just wrote to replace close matches 
replace_matches_in_column(df=test_test1, column='tags', string_to_match="for ")

All done!


In [ ]:

# get all the unique values in the 'Country' column
tagss= test_test1['tags'].unique()

# sort them alphabetically and then take a closer look
tagss.sort()
tagss

array(['2pieceset,Shorts,crop top,Sleeve,Summer,tiktok,Spring,Tops,Spaghetti Strap,short sleeves,Outfits,Fashion',
       "90sfashion,Summer,Fashion,Princess,cute,Kawaii,Shirt,comfy,Leisure,short sleeves,Funny,Women's Fashion,women t shirt,Vintage,Cartoons,Women,Harajuku,Casual,T Shirts,Cotton",
       "Beach Shorts,shorts feminino,Waist,pants,Short pants,Slim Fit,summer shorts,women shorts,high waist,Women's Fashion,Shorts,high waisted shorts,shortsforwomen,Casual,Women",
       "Bikini,Fashion,womenbikiniset,bikini set,pushupsuit,women swimsuit,New arrival,Swimwear,Swimsuit,padded,women swimwear,Bras,Suits,women bathing suit,Women,Women's Fashion,Padded Bra,Swimming",
       "Blues,Summer,Plus Size,Tank,Sexy Top,letter print,Workout tanktop,Tops,Grey,tank top,Women's Fashion,Fashion,women t shirt,Women,Casual T-Shirt,Print,sleeveless,white,women top,T Shirts,Cotton",
       "Blues,Summer,Shorts,Shirt,Sleeve,cottonlinenshirt,Tops,short sleeves,casual shirt,henley,Casual,T Shirts,Green

#Applying label encoder.

In [ ]:
le = preprocessing.LabelEncoder()#converting categorical value to numeric value.
test_test1['tags'] = le.fit_transform(test_test1.tags.values)
test_test1['tags'].dtypes

dtype('int64')

In [ ]:
test_test1.shape

(479, 32)

In [ ]:
test_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479 entries, 937 to 196
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         479 non-null    float64
 1   retail_price                  479 non-null    int64  
 2   currency_buyer                479 non-null    object 
 3   units_sold                    479 non-null    int64  
 4   uses_ad_boosts                479 non-null    int64  
 5   rating_count                  479 non-null    int64  
 6   badges_count                  479 non-null    int64  
 7   badge_local_product           479 non-null    int64  
 8   badge_product_quality         479 non-null    int64  
 9   badge_fast_shipping           479 non-null    int64  
 10  tags                          479 non-null    int64  
 11  product_color                 479 non-null    int64  
 12  product_variation_size_id     479 non-null    int64  
 13  pro

In [ ]:
test_test1=test_test1.drop(columns=['currency_buyer','urgency_text','origin_country','merchant_title','merchant_name','merchant_info_subtitle','merchant_has_profile_picture','merchant_profile_picture','merchant_id','theme','crawl_month'],axis=1)
# drop usless columns

In [ ]:
test_test1.shape

(479, 21)

In [ ]:
test_test1.isnull()

,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,merchant_rating_count,merchant_rating
id,,,,,,,,,,,,,,,,,,,,,
937,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
270,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
308,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
913,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
403,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1416,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
63,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
test_test1=test_test1.fillna(0)# fill nan value

In [ ]:
test_pred1=clf1.predict(test_test1)# prediction of model
print(test_pred1)
# test_pred=test_pred.astype(float)
# test_pred

[2. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 2. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5.
 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 2. 2. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 2. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2.

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- merchant_rating
Feature names seen at fit time, yet now missing:
- rating

  warnings.warn(message, FutureWarning)


#Saving result of prediction in csv file

In [ ]:
_id = test_test1.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred1}))
dataframe.to_csv('dt2_afaf.csv', index=False)

#Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
clf1= GaussianNB()
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}
clf1 = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)



In [ ]:
clf1.fit(X_train,y_train)#fit model

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  UserWarning,


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


GridSearchCV(cv=10, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.848035...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [ ]:
Y_pred=clf1.predict(X_train)

In [ ]:
print(f1_score(y_train, Y_pred, average="weighted"))


0.6045526541077163


In [ ]:
Y_test=clf1.predict(X_test)

In [ ]:
print("f1_score:",metrics.f1_score(y_test,Y_test, average="weighted" ))

f1_score: 0.5462555066079294


In [ ]:
test_pred=clf1.predict(test_test1)
test_pred

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- merchant_rating
Feature names seen at fit time, yet now missing:
- rating

  warnings.warn(message, FutureWarning)


array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [ ]:
_id = test_test1.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred1}))
dataframe.to_csv('naive2_afaf.csv', index=False)

Support vector machine

In [ ]:

#Import svm model
from sklearn import svm

#Create a svm Classifier
clf1 = svm.SVC(C=.1,kernel="rbf") # Linear Kernel. Tunning the important hyperparamter of svm




In [ ]:

#Scale data

scaler = preprocessing.StandardScaler().fit(X_train
    )

# mean values for each column
print(scaler.mean_)

[8.32810282e+00 2.32729498e+01 4.63621297e+03 4.39412485e-01
 4.00734394e+00 9.63977968e+02 1.22399021e-01 2.20318237e-02
 8.69033048e-02 1.34638923e-02 4.44474908e+02 1.75055080e+01
 5.09339045e+01 3.25104039e+01 4.67809058e+01 2.33170135e+00
 1.22399021e-03 4.06572827e+01 4.98310894e+01 3.17013464e-01
 2.63036952e+04]


In [ ]:
X_train_preprocessed = scaler.transform(
    X_train)

# zero mean (or very close to zero) for each column
print('scaled data should have zero mean', X_train_preprocessed.mean(axis=0))

scaled data should have zero mean [-1.30454603e-17  3.75056983e-17 -5.97916929e-17 -5.87045712e-17
  1.63068253e-16  2.60909205e-17  2.82651639e-17 -6.08788146e-17
 -1.08712169e-17 -5.65303278e-17  6.95757881e-17  4.56591109e-17
  9.13182219e-17 -5.21818411e-17 -8.04470050e-17 -1.78287957e-16
  3.26136507e-17  4.13106242e-17  1.24040585e-15  3.04394073e-17
  1.73939470e-17]


In [ ]:
# zero std (or very close to 1) for each column
print('scaled data should have 1 std',X_train_preprocessed.std(axis=0))

scaled data should have 1 std [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
#Apply scaling.


scaler = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(X_train
    )
# here we specified the range as 0 to 1

In [ ]:
X_train_preprocessed = scaler.transform(
    X_train)

# zero min for each column
print(X_train_preprocessed.min(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# max one for each column
print(X_train_preprocessed.max(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
 # apply  'l2' normliztion



scaler = preprocessing.Normalizer(norm='l2').fit(
    X_train)

In [ ]:
X_train_preprocessed= scaler.transform(
    X_train )

In [ ]:
# unit l2 norm
from numpy import linalg as LA
LA.norm(X_train_preprocessed, ord=2, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
clf1.fit(X_train_preprocessed,y_train)  #fit model

SVC(C=0.1)

In [ ]:
Y_pred=clf1.predict(X_train_preprocessed) 


Measure performance metrics.

In [ ]:
print(f1_score(y_train, Y_pred, average="weighted"))

0.6007821681016295


In [ ]:
print("f1_score",metrics.f1_score(y_test,Y_test, average="weighted" ))

f1_score 0.5462555066079294


In [ ]:
test_pred1=clf1.predict(test_test1)
test_pred1

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

# Saving result of prediction in file .csv

In [ ]:
_id = test_test1.index
dataframe=pd.DataFrame(pd.DataFrame(data={'id': np.asarray(_id), 'rating': test_pred1}))
dataframe.to_csv('svm2.afaf.csv', index=False)

## Questions

Why Data Mining is a misnomer? What is another preferred name?

* The term "data mining" is a misnomer, because the goal is the extraction of patterns and knowledge from large amounts of data, not the extraction (mining) of data itself.Knowledge discovery (mining) in databases (KDD).

What is the general knowledge discovery process? What is the difference between a data engineer and data scientist/AI engineer?

 The general KDD process:
1. Selecting the goals of the knowledge discovery process.
2. Selecting of data for knowledge . 
3. Data pre-processing. 
4. Applying the data mining methods. 
---
**Data Engineers** are data professionals who are preparing big data infrastructure to be analysed by data scientists.
**Data Scientists** are responsible to turn raw data into intelligence and knowledge. They are applying statistics, machine learning and analytic approaches to solve business or scientific problems













**In classification**, data is grouped into categories based on a training dataset.

**In prediction**, a classification/regression model is built to predict the outcome(continuous value).

Why data science/machine learning is a bad idea in the context of information security?

* the naive transfer of machine learning principles to the domain of security needs to be taken with caution. Machine learning was not designed with security in mind and as such is prone to adversarial manipulation and reverse engineering. 
* Increased risk of data breach and fine
* Increased uncertainty
* Difficult to evaluate change in AI
* Difficult to verify against compliance
* Responsibility, accountability, liability
*Should be part of the risk management framework


What is CIA principle and how can we use it to access the security/privacy aspect of the AI system/pipelines?

**Confidentiality**:Typically, this involves ensuring that only those who are authorized have access to specific assets and that those who are unauthorized are actively prevented from obtaining access.

Integrity: is about ensuring that data has not been tampered with and, therefore, can be trusted. It is correct, authentic, and reliable.

**Availability**:means that networks, systems, and applications are up and running. It ensures that authorized users have timely, reliable access to resources when they are needed.

**how can we use it to access the security/privacy aspect of the AI system/pipelines?**
 one of these three principles might take precedence over another. For example, confidentiality is vital within intelligence services.
  CIA , is a model designed to guide policies for information security within an organization asMachine learning was not designed with security.